# Evaluate reasoning datasets with `logikon` debuggers

Use this notebook to

* debug a reasoning dataset with `logikon`
* customize `logikon` configuration (e.g., expert_model, debuggers)
* upload debugging results (artifacts and metrics) to huggingface hub

### Set-up

In [40]:
import getpass
import os

GH_ACCESS_TOKEN = getpass.getpass("GH_ACCESS_TOKEN: ")
if GH_ACCESS_TOKEN:
    os.environ["GH_ACCESS_TOKEN"] = GH_ACCESS_TOKEN

OPENAI_API_KEY = getpass.getpass("OPENAI_API_KEY: ")
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

HUGGINGFACEHUB_API_TOKEN = getpass.getpass("HUGGINGFACEHUB_API_TOKEN: ")
if HUGGINGFACEHUB_API_TOKEN:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [2]:
# install logikon module from github repo

# hatch packaging
%pip install hatch

# for using local models with vllm / example 3:
%pip install -U "vllm>=0.1.7" git+https://$GH_ACCESS_TOKEN@github.com/logikon-ai/logikon.git
%pip install "tokenizers==0.13.3"
%pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 80.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 107.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 17.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-

In [4]:
%pip list | grep pydantic

pydantic                          1.10.12
Note: you may need to restart the kernel to use updated packages.


### Imports, data models, configs

In [30]:
import dataclasses
import datetime
import json
import os
import pprint
import tempfile
import uuid

import datasets
import huggingface_hub as hf_hub
import logikon
import networkx as nx


In [3]:
@dataclasses.dataclass
class ReferenceDataset:
    """Reference dataset metadata."""
    path: str
    id_field: str = None
    split: str = None
    revision: str = None

@dataclasses.dataclass
class EvalResultRecord:
    """Evaluation result record."""
    id: str
    reference_id: str
    reference_dataset: dict
    lang: str
    created_date: str
    argmap: dict
    scores: dict
    model: str
    metadata: dict

In [4]:
REASONING_DATASET = dict(
    path='logikon/oasst1-delib',
    split='validation',
    revision=None,
    id_field='message_id',
)

EVALRESULTS_DATASET_PATH = 'logikon/delib-evals'

LANG = "en"

DEBUG_CONFIG = dict(
    artifacts=["networkx_graph"],
    metrics=["argmap_size", "argmap_avg_katz_centrality","argmap_attack_ratio"],
    expert_model="Open-Orca/OpenOrca-Platypus2-13B",
    llm_framework="VLLM",
    #expert_model="gpt-3.5-turbo-instruct",
    #llm_framework="OpenAI",
)

In [47]:
reference_dataset = ReferenceDataset(**REASONING_DATASET)
config = logikon.DebugConfig(**DEBUG_CONFIG)
evalresults_dataset = dict(
    path=EVALRESULTS_DATASET_PATH,
    filename=f"{config.expert_model.replace('/','_')}_{reference_dataset.split}.jsonl",
    subfolder=os.path.join("data",reference_dataset.path),
)

In [48]:
print(reference_dataset)
print(config)
print(evalresults_dataset)

ReferenceDataset(path='logikon/oasst1-delib', id_field='message_id', split='validation', revision=None)
expert_model='Open-Orca/OpenOrca-Platypus2-13B' expert_model_kwargs={'temperature': 0.7} llm_framework='VLLM' generation_kwargs=None inputs=[] metrics=['argmap_size', 'argmap_avg_katz_centrality', 'argmap_attack_ratio'] artifacts=['networkx_graph'] report_to=[]
{'path': 'logikon/delib-evals', 'filename': 'Open-Orca_OpenOrca-Platypus2-13B_validation.jsonl', 'subfolder': 'data/logikon/oasst1-delib'}


### Load reference dataset

In [7]:
# login to huggingface hub
hf_hub.login(os.environ["HUGGINGFACEHUB_API_TOKEN"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
# load the reference dataset
ref_ds = datasets.load_dataset(
    reference_dataset.path,
    split=reference_dataset.split,
    revision=reference_dataset.revision,
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/128 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

In [9]:
# optional: check ref dataset features
# ref_ds.features

In [10]:
assert reference_dataset.id_field in ref_ds.features, f"Reference dataset must have {reference_dataset.id_field} field."

In [60]:
os.path.join(evalresults_dataset["subfolder"], evalresults_dataset["filename"])

'data/logikon/oasst1-delib/Open-Orca_OpenOrca-Platypus2-13B_validation.jsonl'

In [61]:
# download the eval results dataset (if exists)
eval_results = []

if hf_hub.file_exists(
    evalresults_dataset["path"], os.path.join(evalresults_dataset["subfolder"], evalresults_dataset["filename"]),
    repo_type="dataset",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
):
        
    eval_file = hf_hub.hf_hub_download(
        evalresults_dataset["path"],
        filename=evalresults_dataset["filename"],
        subfolder=evalresults_dataset["subfolder"],
        repo_type="dataset"
    )

    with open(eval_file) as f:
        for line in f:
            record = EvalResultRecord(**json.loads(line))
            eval_results.append(record)

    print(f"Loaded {len(eval_results)} eval results.")
    


Loaded 3 eval results.


In [62]:
# iterate over the reference dataset:
# - compute the metrics for records not in the eval results dataset
# - update the eval results dataset

for example in ref_ds.select(list(range(6))):

    if any(r.reference_id == example[reference_dataset.id_field] for r in eval_results):
        continue

    debug_results = logikon.score(
        prompt=example["history"],
        completion=example["text"],
        config=config
    )

    argmap = next((a.data for a in debug_results.artifacts if a.id == "networkx_graph"), None)
    argmap = nx.node_link_data(argmap) if argmap else {}

    scores = {
        "id": [],
        "value": [],
        "comment": [],
    }
    for score in debug_results.scores:
        for key in scores:
            scores[key].append(score.__getattribute__(key))
    scores["names"] = scores.pop("id")
    scores["values"] = scores.pop("value")
    scores["comments"] = scores.pop("comment")

    eval_record = EvalResultRecord(
        id = str(uuid.uuid4()),
        reference_id=example[reference_dataset.id_field],
        reference_dataset=dataclasses.asdict(reference_dataset),
        lang=example.get("lang", LANG),
        created_date=datetime.datetime.now().isoformat(),
        argmap=argmap,
        scores=scores,
        model=config.expert_model,
        metadata={"logikon_config": config.dict(), "logikon_version": logikon.__version__},
    )

    eval_results.append(eval_record)



> Entering new ClaimExtractionChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment

Identify the basic question addressed in a text.

# Detailed Instructions

Use the inputs (a TEXT) to solve your assignment and read the following instructions carefully.
Identify the overarching question the TEXT raises and addresses.
Note that the overarching question is not necessarily explicitly stated in the TEXT (as shown in some examples below). It may be implicit. And it may deviate from any explicitly stated questions or instructions.
State a single bold question in a concise and most-simple way. Use plain language.
Don't provide alternatives, comments or explanations.

# Example

TEXT:
:::
Should we visit Mars?
- Mars is a planet and rather difficult to reach.
- Visiting Mars would be a great 

Processed prompts: 100%|██████████| 2/2 [00:15<00:00,  7.61s/it]



> Finished chain.
> Answer: The text's strongest reason against the claim is:

"The main advantage of two-factor authentication is increased login security. However, some shortcomings include an increase in the time it takes to log into a system and the risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)2."

This quote highlights the pros and cons of two-factor authentication, acknowledging its benefits (increased login security) while also mentioning potential drawbacks (longer login time and risk of losing physical media).

Please note that the provided quote may not directly contradict the claim but offers counterpoints to consider.

1: The given text does not contain a reason against the claim, as it simply explains the advantages and disadvantages of two-factor authentication.
2: The provided quote discusses advantages and disadvantages, not reasons against the claim.
3: The provided text does not contain a reason against the

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.03it/s]



> Finished chain.
> Answer: (A)


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment

State the two opposite answers to a TEXT's overarching binary question.

# Inputs

Use the following inputs (a QUESTION and a TEXT) to solve your assignment.

QUESTION:
:::
The text's strongest reason against the claim is:

"The main advantage of two-factor authentication is increased login security. However, some shortcomings include an increase in the time it takes to log into a system and the risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)2."

This quote highlights the pros and cons of two-factor authentication, acknowledging its benefits (increased login security) while also mentioning potential drawbacks (longer login time and risk of losing physical med

...


> Finished chain.
> Answer: 
The main advantage of two-factor authentication is increased login security; however, some shortcomings include an increase in the time it takes to log into a system.

The strongest reason against the claim is: "an increase in the time it takes to log into a system."

Note: The text's reason is not necessarily a "con" itself, but it suggests that one of the disadvantages of two-factor authentication is the increase in login time.

The provided quote doesn't necessarily disagree with the claim; it simply states one of the potential drawbacks of implementing two-factor authentication. The claim's subject matter, "Con: Increase in login time," suggests a negative aspect of the claim, which aligns with the provided text's reason.

The text provides a reason against the claim that the increase in login time is a shortcoming of two-factor authentication. This demonstrates that the text recognizes this potential drawback, providing support for the claim. The text

Processed prompts: 100%|██████████| 1/1 [00:22<00:00, 22.04s/it]



> Finished chain.
> Answer: 
The risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)2.

The text's strongest reason for the claim is found in the quote: "The risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)2." This directly addresses the mentioned con (risk of losing physical media used for authentication).

Sentence number two is used to provide a relevant example of the type of physical media (e.g., a mobile phone or U2F key) that might be lost. The text doesn't explain why this might be a problem or list other potential cons, as requested, but it does give a clear reason for the claim.

There is no need to include the first part of the text, as it is not directly related to the con. The second part of the text is a question asking about specific authentication methods, which does not provide a reason for the claim.

This answer is focused on the relevant text and the cla

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.80s/it]


> Finished chain.
> Answer: 
The text's strongest reason against the claim is: "The main advantage of two-factor authentication is increased login security. However, some shortcomings include an increase in the time it takes to log into a system and the risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)."

This clearly states that one disadvantage of two-factor authentication is the risk of losing physical media used for authentication.

Please note that the text's strongest reason is the risk of losing the physical media, not the increase in time it takes to log into a system.

Original text: "the risk of losing physical media used for one of the authentication steps (such as a mobile phone or U2F key)"

Modified text: "the risk of losing physical media used for authentication"

Final answer: "the risk of losing physical media used for authentication"

Please verify that this matches the claim you'd like to address.

Yes, this m

In [63]:
for record in eval_results:
    print(json.dumps(dataclasses.asdict(record)))

{"id": "9bd3b97a-c70a-4876-a2c0-f5d821dbd04c", "reference_id": "c743da26-28dd-45fc-8a52-e1bd500ca8a2", "reference_dataset": {"path": "logikon/oasst1-delib", "id_field": "message_id", "split": "validation", "revision": null}, "lang": "en", "created_date": "2023-09-27T12:32:31.214534", "argmap": {"directed": true, "multigraph": false, "graph": {}, "nodes": [{"text": "TensorFlow", "label": "Claim-1", "annotations": [], "nodeType": "proposition", "id": "84f87cb0-52cd-4a48-9b6c-928a7d87869d"}, {"text": "PyTorch", "label": "Claim-2", "annotations": [], "nodeType": "proposition", "id": "1aeff1d9-8ee8-4d4b-a171-ed41f72da9b8"}, {"text": "Keras", "label": "Claim-3", "annotations": [], "nodeType": "proposition", "id": "4e84f3c5-029c-4f82-a679-e00f2baf4b1c"}, {"text": "MXNet", "label": "Claim-4", "annotations": [], "nodeType": "proposition", "id": "625dd687-2eeb-4777-8c94-3c6c0288970b"}, {"text": "Caffe", "label": "Claim-5", "annotations": [], "nodeType": "proposition", "id": "a27142c3-00d2-4ebf-8

In [64]:
# save the eval results dataset

with tempfile.NamedTemporaryFile() as tmpf:
    for record in eval_results:
        tmpf.write(json.dumps(dataclasses.asdict(record)).encode("utf-8"))
        tmpf.write("\n".encode("utf-8"))
    tmpf.flush()
    
    api = hf_hub.HfApi()
    api.upload_file(
        path_or_fileobj=tmpf.name,
        path_in_repo=os.path.join(evalresults_dataset["subfolder"],evalresults_dataset["filename"]),
        repo_id=evalresults_dataset["path"],
        repo_type="dataset",
        token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    )


In [65]:
# check

In [66]:
ds_test = datasets.load_dataset(evalresults_dataset["path"])
ds_test

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'reference_id', 'reference_dataset', 'lang', 'created_date', 'argmap', 'scores', 'model', 'metadata'],
        num_rows: 6
    })
})

In [67]:
ds_test["validation"].features

{'id': Value(dtype='string', id=None),
 'reference_id': Value(dtype='string', id=None),
 'reference_dataset': {'path': Value(dtype='string', id=None),
  'id_field': Value(dtype='string', id=None),
  'split': Value(dtype='string', id=None),
  'revision': Value(dtype='null', id=None)},
 'lang': Value(dtype='string', id=None),
 'created_date': Value(dtype='string', id=None),
 'argmap': {'directed': Value(dtype='bool', id=None),
  'multigraph': Value(dtype='bool', id=None),
  'graph': {},
  'nodes': [{'text': Value(dtype='string', id=None),
    'label': Value(dtype='string', id=None),
    'annotations': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None),
    'nodeType': Value(dtype='string', id=None),
    'id': Value(dtype='string', id=None)}],
  'links': [{'valence': Value(dtype='string', id=None),
    'source': Value(dtype='string', id=None),
    'target': Value(dtype='string', id=None)}]},
 'scores': {'names': Sequence(feature=Value(dtype='string', id=None), length=-1, i